In [1]:
# importing required libraries
import pandas as pd
import json

```
{
        "textbook-paragraph": "{Error Correction}\n\nDetection of errors is importantn\n",
        "GPT-3-RLHF-Generations": {
            "question": " How can a",
            "answers": [
                " A digital to correct acurred.",
                " Error bit was in error and correct it accordingly.",
                " Hamming codes any two bit errors."
            ]
        }
    },
```

In [7]:
# load json from file (direct from Neha)
with open('RLHF_Version3_set2.json') as f:
  data = json.load(f)

# reformat
new_data = []
for el in data:
  newdict = {
    'textbook-paragraph': el['textbook-paragraph'],
    'question': el['GPT-3-RLHF-Generations']['question'],
    'answers': el['GPT-3-RLHF-Generations']['answers'],
  }
  new_data.append(newdict)
  new_data[0]

# to csv
df = pd.read_json(json.dumps(new_data))
df

{'textbook-paragraph': "{Error Correction}\n\nDetection of errors is important, but may sometimes not be enough.\nWhat can a digital system do when it detects an error?  In some\ncases, the system may be able to find the original value elsewhere, or\nmay be able to re-compute the value from other values.  In other \ncases, the value is simply lost, and the digital system may need\nto reboot or even shut down until a human can attend to it.\n\nMany real systems cannot afford such a luxury.  Life-critical systems\nsuch as medical equipment and airplanes should not turn themselves off\nand wait for a human's attention.  Space vehicles face a similar dilemma,\nsince no human may be able to reach them.\n\nCan we use a strategy similar to the one that we have developed for error\ndetection in order to try to perform { error correction}, recovering\nthe original value?  Yes, but the overhead---the\nextra bits that we need to provide such functionality---is higher.\n\n\n\nLet's start by thinki

In [14]:
# Reformat for Scale.ai upalod
# [{"type": "text", "content": "#### Option 1\n-------\n**Answer 1**"}, {"type": "text", "content": "#### Option 2\n-------\n**Answer 2**"}, {"type": "text", "content": "#### Option 3\n-------\n**Answer 3**"}]
def json_gen(df):
  upload_entry = []
  meta_data = []
  count = 0
  for quest, ans in zip(df['question'], df['answers']):
    count += 1
    entry = [
        {
            "type": "text",
            "content": quest.split("?")[0] + "?\n" + "#### Option 1\n-------\n" + ans[0].strip()
        },
        {
            "type": "text",
            "content": "#### Option 2\n-------\n" + ans[1].strip()
        },
        {
            "type": "text",
            "content": "#### Option 3\n-------\n" + ans[2].strip()
        },
    ]
    # {"type": "text", "content": "#### Option 4\n-------\n" +  ans[3].strip()}]
    upload_entry.append(json.dumps(entry, indent=1))
    meta_data.append(json.dumps({'id': count}, indent=1))

  data = pd.DataFrame()
  data['attachments_json'] = upload_entry
  data['metadata'] = meta_data
  return data


# RUN IT
rlhf_data = json_gen(df)
rlhf_data.to_csv('rlhf_v3_set2_for_upload_to_scale.csv', index=False)
rlhf_data

,attachments_json,metadata
0,"[\n {\n ""type"": ""text"",\n ""content"": "" How c...","{\n ""id"": 1\n}"
1,"[\n {\n ""type"": ""text"",\n ""content"": "" How c...","{\n ""id"": 2\n}"
2,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 3\n}"
3,"[\n {\n ""type"": ""text"",\n ""content"": "" How a...","{\n ""id"": 4\n}"
4,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 5\n}"
...,...,...
101,"[\n {\n ""type"": ""text"",\n ""content"": "" How i...","{\n ""id"": 102\n}"
102,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 103\n}"
103,"[\n {\n ""type"": ""text"",\n ""content"": "" How i...","{\n ""id"": 104\n}"
104,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 105\n}"


IndexError: list index out of range

In [7]:
# loaing test data
from datasets import load_dataset
test_dataset = load_dataset("kastan/EE_QA_for_RLHF")
test_dataset['train']

# test
test_dataframe = pd.DataFrame()
test_dataframe['textbook-paragraph'] = test_dataset['train']['textbook-paragraph']
test_dataframe['question'] = test_dataset['train']['question']
test_dataframe['answers'] = test_dataset['train']['answers']
test_dataframe.head()

Using custom data configuration kastan--EE_QA_for_RLHF-971999be283e776c
Found cached dataset json (/home/sbdesai2/.cache/huggingface/datasets/kastan___json/kastan--EE_QA_for_RLHF-971999be283e776c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 473.67it/s]


,textbook-paragraph,question,answers
0,"{Finite State Machine Design Examples, Part I}...",What is a finite state machine?\n,[\n\nA finite state machine is a mathematical ...
1,"{Finite State Machine Design Examples, Part I}...",What are the basic steps of designing a finit...,[ The basic steps of designing a finite state ...
2,"{Finite State Machine Design Examples, Part I}...",What is an example of a design that is more i...,[ A counter with a number of states that is no...
3,"{Finite State Machine Design Examples, Part I}...",What is the design process for a digital FSM?\n,[ The design process for a digital FSM is a pr...
4,"{Finite State Machine Design Examples, Part I}...",What is an example of a counter with a number ...,[ A counter with a number of states that is no...


In [8]:
# loading the rlhf data
f = open('/mnt/project/chatbotai/saish/data/RLHF_data_v2_1.json')
rlhf_dataset = json.load(f)

para, q, a = [],[],[]
for ele in rlhf_dataset:
    para.append(ele['textbook-paragraph'])
    q.append(ele['GPT-3-RLHF-Generations']['question'])
    a.append(ele['GPT-3-RLHF-Generations']['answers'])

rlhf_dataframe = pd.DataFrame()
rlhf_dataframe['textbook-paragraph'] = para
rlhf_dataframe['question'] = q
rlhf_dataframe['answers'] = a


rlhf_dataframe.head()


,textbook-paragraph,question,answers
0,"{Finite State Machine Design Examples, Part I}...",What is a finite state machine?\n,[ A finite state machine (FSM) is a model of c...
1,"{Finite State Machine Design Examples, Part I}...",What are the basic steps of designing a finit...,[ The basic steps of designing a finite state ...
2,"{Finite State Machine Design Examples, Part I}...",What is an example of a design that is more i...,[ A design that is more intuitive and easier f...
3,"{Finite State Machine Design Examples, Part I}...",What is the design process for a digital FSM?\n,[ The design process for a digital FSM typical...
4,"{Finite State Machine Design Examples, Part I}...",What is an example of a counter with a number ...,[ An example of a counter with a number of sta...


In [11]:
test_data = json_gen(test_dataframe)
rlhf_data = json_gen(rlhf_dataframe)

In [15]:
test_data

,attachments_json,metadata
0,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 1\n}"
1,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 2\n}"
2,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 3\n}"
3,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 4\n}"
4,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 5\n}"
...,...,...
175,"[\n {\n ""type"": ""text"",\n ""content"": ""What d...","{\n ""id"": 176\n}"
176,"[\n {\n ""type"": ""text"",\n ""content"": ""What i...","{\n ""id"": 177\n}"
177,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 178\n}"
178,"[\n {\n ""type"": ""text"",\n ""content"": "" What ...","{\n ""id"": 179\n}"


In [14]:
rlhf_data.head

<bound method NDFrame.head of                                       attachments_json          metadata
0    [\n {\n  "type": "text",\n  "content": " What ...    {\n "id": 1\n}
1    [\n {\n  "type": "text",\n  "content": " What ...    {\n "id": 2\n}
2    [\n {\n  "type": "text",\n  "content": " What ...    {\n "id": 3\n}
3    [\n {\n  "type": "text",\n  "content": "What i...    {\n "id": 4\n}
4    [\n {\n  "type": "text",\n  "content": "What i...    {\n "id": 5\n}
..                                                 ...               ...
369  [\n {\n  "type": "text",\n  "content": " How d...  {\n "id": 370\n}
370  [\n {\n  "type": "text",\n  "content": " What ...  {\n "id": 371\n}
371  [\n {\n  "type": "text",\n  "content": " What ...  {\n "id": 372\n}
372  [\n {\n  "type": "text",\n  "content": " How d...  {\n "id": 373\n}
373  [\n {\n  "type": "text",\n  "content": " What ...  {\n "id": 374\n}

[374 rows x 2 columns]>

In [18]:
test_data.to_csv("test_upload_ranking.csv", index = False)

In [19]:
rlhf_data.to_csv("upload_ranking.csv", index = False)